In [1]:
import flymovie as fm
import numpy as np
import scipy.ndimage as ndi
from importlib import reload
from flymovie.general_functions import extract_box, stack_normalize_minmax

%load_ext autoreload
%autoreload 2

In [52]:
stack = fm.read_czi('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/psf/2022-02-09/tetraspeck-field1-100x50.czi', swapaxes=False)

def extract_beads(stack, thresh, box_dims):
    """Extract windows centered at beads."""

    # Make mask with thresholding, use opening to get rid of small objects, label.
    mask = np.where(stack > thresh, 1, 0)
    mask = ndi.morphology.binary_opening(mask, structure=np.ones((3,3,3)))

    # Strategy: ndi.measurements.center_of_mass finds the center of mass of objects
    # in a labelmask. I'm worried that if I use the thresholded mask, subtle
    # differences in thresholding might change the center. It would be better
    # to determine the center of mass within a nice window around each object. 
    # To do this, first expand the objects using morphological
    # dilation and find the center of mass of these larger objects.

    mask = ndi.morphology.binary_dilation(mask, structure=np.ones((4,8,8)))
    lmask,_ = ndi.label(mask)

    # Find center of mass for each spot (within window defined above).
    centers = ndi.measurements.center_of_mass(stack, lmask, np.arange(1, np.max(lmask) + 1))

    boxes = np.ndarray(tuple([0]) + box_dims)
    for center in centers:
            center = [round(x) for x in center]
            box = extract_box(stack, center, box_dims, pad=False)
            # If the box extends beyond the image borders, box returned by extract_box
            # will be smaller than box_dims (because padding = false).
            if not np.array_equal(box.shape, box_dims):
                continue
            box = stack_normalize_minmax(box)
            box = np.expand_dims(box, axis=0)
            boxes = np.vstack([boxes, box])
    return boxes

def extract_beads_batch(stacklist, thresh, box_dims):
    boxes = np.ndarray(tuple([0]) + box_dims)
    for stack in stacklist:
        boxes = np.vstack([boxes, extract_beads(stack, thresh, box_dims)])
    return boxes

def remove_bad_beads(stack, bad_indexes):
    mask = np.ones(stack.shape[0])
    mask[bad_indexes] = 0
    mask = mask.astype(bool)
    return(stack[mask])

In [2]:
stack1 = fm.read_czi('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/psf/2022-02-09/tetraspeck-100x50-field4-1AU-green-01.czi', swapaxes=False)
stack2 = fm.read_czi('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/psf/2022-02-09/tetraspeck-100x50-field5-1AU-green-01.czi', swapaxes=False)

stacklist = [stack1[0], stack1[1], stack2[0], stack2[1], stack2[2], stack2[3], stack2[4]]

In [4]:
boxes = fm.extract_beads_batch(stacklist, 20, (21, 21,21))

(2.8159239715998563, 53.35450451694892, 513.4732160123767)
(10.145833333333334, 67.89739884393063, 46.366449903660886)
(30.782759640990236, 89.96748200019726, 709.6650261366999)
(30.692593210442904, 74.05292351322045, 591.7491450496289)
(31.4891104815864, 87.58091784702549, 124.43620396600566)
(30.907060761484033, 112.7421474298978, 470.0465164164576)
(31.20556013462463, 130.9437117933924, 421.61147743663713)
(32.2501276533664, 141.45755829503733, 24.462372650570185)
(31.69061950933819, 5.5994013145051085, 234.78468471399754)
(30.781592920353983, 87.41458407079647, 365.0402477876106)
(30.354692581694284, 117.95373238345981, 670.7753987920087)
(31.390858796890683, 141.16482644319427, 203.41165760171805)
(31.63519301211609, 9.274894336432798, 294.1018033248802)
(31.031754464833053, 22.079570068249623, 404.65826147370143)
(31.14355404672924, 25.57316064245571, 466.6539811143126)
(30.893815635939323, 51.11841306884481, 510.7202800466744)
(31.48389652761594, 54.925075739920764, 294.39095781

In [5]:
fm.viewer([boxes.max(axis=(1)) * 1000, boxes.max(axis=(2)) * 1000], 4)

interactive(children=(Dropdown(description='Color', index=5, options=('gray', 'plasma', 'gray_r', 'Gators', 'G…

In [6]:
bad_beads = [0,31,60,62,74]

In [8]:
boxes_good = fm.remove_bad_beads(boxes, bad_beads)
psf = boxes_good.mean(axis=0)
fm.viewer(psf * 1000, 5)

interactive(children=(Dropdown(description='Color', index=5, options=('gray', 'plasma', 'gray_r', 'Gators', 'G…

In [72]:
psf_file = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/results/psf_20220210.pkl'
fm.save_pickle(psf, psf_file)